# Amazon SageMaker ノートブックインスタンスで AWS Glue と Amazon Athena を用いたデータの前処理

本ハンズオンでは Amazon SageMaker で機械学習モデルを学習させるためのデータを、Amazon S3、Amazon Athena、AWS Glue をSageMakerと連携させて前処理を行う場合についてご体験頂きます。

---
### 実施内容

1. [データの準備](#1.データの準備)
1. [Athenaを使ったデータの確認](#2.Athenaを使ったデータの確認)
1. [Glueによるデータの前処理の実施](#3.Glueによるデータの前処理の実施)
1. [モデルの学習](#4.モデルの学習)
1. [後片付け](#5.後片付け)
---

In [ ]:
from datetime import datetime

import boto3
import pandas

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input

role = get_execution_role()
sess = sagemaker.Session()

## 1.データの準備

今回は [TLC Trip Record Data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page) にある[2019年6月の Green Taxi Trip Records(CSV)](https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2019-06.csv) を利用します。このデータを今回のユースケースである、ノートブックでの展開が困難であるほど容量が大きいデータだと仮定し、一度 Amazon S3 へアップロードした上で、ノートブックインスタンスから削除します。作成後、[マネジメントコンソール](https://s3.console.aws.amazon.com/s3/home)で確認してみましょう。

In [ ]:
# データのダウンロード
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2019-06.csv

# 本ハンズオンで活用するバケットの作成
bucket_name='data-wrangler-{0:%Y%m%d-%H%M%S}'.format(datetime.now())
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.create()

# バケットへのデータのアップロード
s3.Object(bucket_name, 'green_tripdata_2019-06.csv').upload_file('green_tripdata_2019-06.csv')
print('s3://{} へデータがアップロードされました。'.format(bucket_name))

# ノートブックインスタンス上でのデータの削除
!rm green_tripdata_2019-06.csv

# 2.Athenaを使ったデータの確認
### Amazon Athena のセットアップ

Athena のデータベースを作成します。

In [ ]:
ath = boto3.client('athena')
database_name='datawrangler'

# データベースの作成
ath.start_query_execution(
    QueryString='CREATE DATABASE {}'.format(database_name),
    ResultConfiguration={'OutputLocation': 's3://' + bucket_name + '/athena/'})

作成したデータベースにテーブルを作成します。 [マネジメントコンソール](https://console.aws.amazon.com/athena/home) で確認してみましょう。

In [ ]:
query_string = """
CREATE EXTERNAL TABLE green_tripdata(
  VendorID string, 
  lpep_pickup_datetime string,
  lpep_dropoff_datetime string,
  store_and_fwd_flag string,
  RatecodeID string,
  PULocationID string,
  DOLocationID string,
  passenger_count int,
  trip_distance double,
  fare_amount double,
  extra double,
  mta_max double,
  tip_amount double,
  tolls_amount double,
  ehail_fee string,
  improvement_surcharge double,
  total_amount double,
  payment_type string,
  trip_type string,
  congestion_surcharge double
  )
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
LOCATION 's3://{}/';
""".format(bucket_name)

ath.start_query_execution(
        QueryString=query_string,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': 's3://' + bucket_name})

### Data Wrangler と Athena を使ったデータ分析
2019年9月、 [Github](https://github.com/awslabs/aws-data-wrangler) に AWS Data Wrangler が公開されました。Data Wranglerは、各種AWSサービスからデータを取得して、コーディングをサポートしてくれるPythonのモジュールです。 PyAthena や boto3、を活用する場合に比べて、接続設定などのコーディングが簡素になるため、より一層データ分析や ETL 処理に集中することが出来ます。

Data Wrangler をインストールします。

In [ ]:
!pip install awswrangler

Athea で使うクエリをこのノートブックインスタンス上から実行してみましょう。

In [ ]:
import awswrangler

# データを確認するためのSQL文
sql = """
SELECT 
    * 
FROM
    green_tripdata
ORDER BY 
    RAND()
LIMIT
    1000
;
"""

session = awswrangler.Session()
df = session.pandas.read_sql_athena(sql=sql,database=database_name)

# 取得したデータの表示
df.head()

S3 にあるデータを Athena やサンプリングされたデータを用いてノートブックインスタンス上の [Pandas](https://pandas.pydata.org/) を使って探索的データ分析を行い、機械学習モデルの学習に必要な前処理や特徴量作成を検討します。その後、実際の処理は Glue 上で実施します。今回のデータの前処理を行うスクリプトを確認しましょう。

# 3.Glueによるデータの前処理の実施

In [ ]:
!cat preprocess.py

今回準備したスクリプトを S3 へアップロードします。Glue のジョブ作成時にそのスクリプトのパスを指定することで実行が可能です。ジョブ作成時の`Role`を指定して下さい。

In [ ]:
# Glue で使用するRoleの指定
role = '使用するRoleを指定して下さい'

# バケットへのデータのアップロード
s3.Object(bucket_name, 'preprocess.py').upload_file('preprocess.py')
print('s3://{} へ Glue のスクリプトがアップロードされました。'.format(bucket_name))

# Glue のジョブを作成します。
glue = boto3.client('glue')
job = glue.create_job(Name='preprocess', Role=role,
                      Command={'Name': 'pythonshell',
                               'ScriptLocation': 's3://{}/preprocess.py'.format(bucket_name),
                               'PythonVersion': '3'})
# 作成したジョブを開始します。
jobrun = glue.start_job_run(JobName = job['Name'],Arguments = {'--bucket_name': bucket_name} )

ジョブの実行状況を確認しましょう。`SUCCEEDED` となったら完了です。

In [ ]:
import time

while True:
    status = glue.get_job_run(JobName=job['Name'], RunId=jobrun['JobRunId'])
    print('Glue のジョブのステータスは現在 {} です。'.format(status['JobRun']['JobRunState']))
    if status['JobRun']['JobRunState'] == 'SUCCEEDED':
        break
    else:
        time.sleep(10)

# 4.モデルの学習

Glue による処理により、S3 のバケット上に `train.csv` と `validation.csv` が作成されています。今回はこのデータをモデルの学習に使います。

In [ ]:
# データのパスを指定
input_train = 's3://{}/train.csv'.format(bucket_name)
input_validation = 's3://{}/validation.csv'.format(bucket_name)

s3_input_train = s3_input(s3_data=input_train, content_type='text/csv')
s3_input_validation = s3_input(s3_data=input_validation, content_type='text/csv')

それでは学習を始めましょう。まず、XGBoost のコンテナの場所を取得します。コンテナ自体は SageMaker 側で用意されているので、場所を指定すれば利用可能です。XGBoostのhyperparameterに関する詳細は [github](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) もチェックしてください。

In [ ]:
container = get_image_uri(boto3.Session().region_name, 'xgboost')
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='reg:linear',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

`job completed`という文字が出たら学習の完了です。このモデルを使って推論の実行を行うことができます。

# 5.後片付け

本ハンズオンで使用したリソースを削除します。今回使ったリソースは下記になります。
- Athena のテーブル
- Glue のジョブ
- Glue のデータベース
- S3 のバケット

SageMakerへ割り当てられているロールによっては削除の権限がない場合があります。その場合には[マネジメントコンソール](https://console.aws.amazon.com/console/home?)から削除して下さい。

Athena のテーブルを削除します。

In [ ]:
query_string = 'DROP TABLE `green_tripdata`;'

ath.start_query_execution(
        QueryString=query_string,
        QueryExecutionContext={'Database': database_name},
        ResultConfiguration={'OutputLocation': 's3://' + bucket_name})

Glue のジョブとデータベースを削除します。

In [ ]:
glue.delete_job(JobName='preprocess')
glue.delete_database(Name=database_name)

S3 のバケットを削除します。

In [ ]:
# 本ハンズオンで活用したバケットの削除
bucket = s3.Bucket(bucket_name)
bucket.delete()